# Polyiamonds

Working ... https://puzzler.sourceforge.net/docs/FAQ.html#

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon

def get_rectified_coord(x, y, z, theta):
    """Get Rectified Coordinates via (x, y, z) and theta (in degree form).
    """
    radian = np.deg2rad(theta)
    if z == 0:
        return [(x + y * np.cos(radian), y * np.sin(radian)), 
                (x + y * np.cos(radian) + 1, y * np.sin(radian)),
                (x + y * np.cos(radian) + np.cos(radian), (y + 1) * np.sin(radian))
               ]
    elif z == 1:
        return [
            (x + y * np.cos(radian) + np.cos(radian), (y + 1) * np.sin(radian)),
            (x + y * np.cos(radian) + np.cos(radian) + 1, (y + 1) * np.sin(radian)),
            (x + y * np.cos(radian) + 1, y * np.sin(radian))
        ]
    else: 
        print(f"Invalid z coord (only 1 or 0)! Found {z}!")
        return []

# The neighbors of the triangle at coordinates (x, y, 0) are:
# {(x, y, 1), (x-1, y, 1), (x, y-1, 1)}
# The neighbors of the triangle at coordinates (x, y, 1) are:
# {(x, y, 0), (x+1, y, 0), (x, y+1, 0)}

def round_point(point):
    return tuple(map(lambda x: round(x, 5), point))

def round_coords(segments):
    return list(map(lambda line: (round_point(line[0]), round_point(line[1])), segments))

def get_border(coords, theta): 
    candidates = set()
    removed_edge = set()
    final_edges = list()
    for (x, y, z) in coords: 
        candidates.add(f"{x}_{y}_{z}")
        if z == 0: 
            if f"{x}_{y}_{1}" in candidates: 
                removed_edge.add(f"{x}_{y}_{0}_{1}") 
                removed_edge.add(f"{x}_{y}_{1}_{2}") 

            if f"{x - 1}_{y}_{1}" in candidates:  
                removed_edge.add(f"{x}_{y}_{0}_{2}")
                removed_edge.add(f"{x - 1}_{y}_{1}_{1}")

            if f"{x}_{y - 1}_{1}" in candidates: 
                removed_edge.add(f"{x}_{y}_{0}_{0}")
                removed_edge.add(f"{x}_{y - 1}_{1}_{0}")

        elif z == 1: 
            if f"{x}_{y}_{0}" in candidates: 
                removed_edge.add(f"{x}_{y}_{1}_{2}")
                removed_edge.add(f"{x}_{y}_{0}_{1}")
                
            if f"{x + 1}_{y}_{0}" in candidates: 
                removed_edge.add(f"{x}_{y}_{1}_{1}")
                removed_edge.add(f"{x + 1}_{y}_{0}_{2}")

            if f"{x}_{y + 1}_{0}" in candidates: 
                removed_edge.add(f"{x}_{y}_{1}_{0}")
                removed_edge.add(f"{x}_{y + 1}_{0}_{0}")
    for (x, y, z) in coords: 
        pos = get_rectified_coord(x, y, z, theta)
        for i in range(3): 
            if f"{x}_{y}_{z}_{i}" in removed_edge: 
                continue 
            else:
                final_edges.append([pos[i % 3], pos[(i + 1) % 3]])
    return final_edges 
            

def organize_segments(segments):
    # 将每个线段转化为点对字典
    point_dict = {}
    for (x1, y1), (x2, y2) in segments:
        point_dict.setdefault((x1, y1), []).append((x2, y2))
        point_dict.setdefault((x2, y2), []).append((x1, y1))

    # 从其中一个点开始重新构建闭合路径
    start_point = segments[0][0]
    current_point = start_point
    organized_path = [start_point]

    while len(organized_path) != len(segments) + 1:
        # 从当前点选择下一个点
        next_point = point_dict[current_point][0]  # 选择尚未访问的下一个点
        # 移动到下一个点
        point_dict[current_point].remove(next_point)
        point_dict[next_point].remove(current_point)
        organized_path.append(next_point)
        current_point = next_point

    return organized_path

def clear_padding(coords):
    """Adjust the shape to border.

    Args:
        coords (_type_): _description_

    Returns:
        _type_: _description_
    """
    min_x = min(x for x, _, _ in coords) 
    min_y = min(y for _, y, _ in coords) 
    offset_x = min_x - 0
    offset_y = min_y - 0 

    if offset_x > 0 or offset_y > 0:
        new_coords = []
        for (x, y, z) in coords: 
            new_coords.append((x - offset_x, y - offset_y, z))
        return new_coords 
    else:
        return coords

def flap_120(coords):
    max_x = max(x for x, _, _ in coords)
    max_y = max(y for _, y, _ in coords)
    axis_ = max(max_x, max_y)
    flap_coords = []
    for (x, y, z) in coords: 
        if z == 1: 
            flap_coords.append((axis_ - y, axis_ - x, 0))
        elif z == 0:
            flap_coords.append((axis_ - y, axis_ - x, 1))
        else:
            print(f"Invalid coords: {z}, should be 0 or 1.")
    return clear_padding(flap_coords) 

def flap_90(coords):
    max_xy = max([ x + y if z == 0 else x + y + 1 for (x, y, z) in coords])
    new_coords = []
    for (x, y, z) in coords: 
        if z == 0: 
            new_coords.append((max_xy - x - y, y, z))
        else:
            new_coords.append((max_xy - x - y - 1, y, z))
    return clear_padding(new_coords)

def flap_30(coords):
    flap_coords = [] 
    for (x, y, z) in coords: 
        flap_coords.append((y, x, z)) 
    return clear_padding(flap_coords)

def all_possible_rotations(coords, opts = ["flap_90", "flap_120", "flap_30"]): 
    final_coords = [coords]
    for idx, opt in enumerate(opts): 
        if opt == "flap_90":
            next_coords = flap_90(coords)
        elif opt == "flap_120": 
            next_coords = flap_120(coords)
        elif opt == "flap_30":
            next_coords = flap_30(coords)
        new_coords = all_possible_rotations(next_coords, opts[: idx] + opts[idx + 1:])
        final_coords = final_coords +  new_coords
    return final_coords

def remove_dup(coords):
    seen = set()
    result = []
    for sublist in coords:
        # 将子列表中的元组排序并转换为元组作为唯一标识
        sorted_sublist = sorted(sublist)
        key = tuple(sorted_sublist)
        if key not in seen:
            seen.add(key)
            result.append(sublist)
    return result

def plot_polyiamond(coords_list):
    
    theta = 60
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    
    for idx, coord in enumerate(coords_list):
        temp_coords = get_border(coord, theta)
        temp_coords = round_coords(temp_coords)
        temp_coords = organize_segments(temp_coords)

        polygon = Polygon(temp_coords, closed=True, fill=True, edgecolor='black', facecolor='skyblue', alpha=0.5)
        ax.add_patch(polygon)
        
        # 设置坐标轴的标签和标题
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_xlim(-1, 7)
    ax.set_ylim(-1, 7)
    ax.grid()
    ax.set_title('Filled Polygon from Line Segments')
    plt.show()

if __name__ == "__main__": 
    triangles = [
        (0, 0, 0), (0, 0, 1), (1, 0, 0), (1, 0, 1), (2, 0, 0), (2, 0, 1),(0, 1, 0), (0, 1, 1),(0, 2, 0), (0, 2, 1),(1, 2, 0), (1, 2, 1)
        
        # (0, 0, 1),
        # (1, 0, 0), (1, 0, 1), 
        # (0, 1, 0), (0, 1, 1),
        # (1, 1, 0), (2, 0, 0)
    ]
    all_coords = all_possible_rotations(triangles)
    # all_coords = []
    # test_coords = []
    # all_coords.append(triangles)
    # # plot_polyiamond([triangles])
    # triangles_120 = flap_120(triangles)
    # all_coords.append(triangles_120)
    # # plot_polyiamond([triangles_120])
    # triangles_30 = flap_30(triangles)
    # all_coords.append(triangles_30)
    # # plot_polyiamond([triangles_30])
    # triangles_120_2 = flap_30(triangles_120)
    # all_coords.append(triangles_120_2)
    # # plot_polyiamond([triangles_120_2])
    # triangles_90 = flap_90(triangles)
    # all_coords.append(triangles_90)
    # # plot_polyiamond([triangles_90])
    # triangles_30_2 = flap_120(triangles_30)
    # all_coords.append(triangles_30_2)
    # # print(triangles_30_2)
    # all_coords = remove_dup(all_coords)
    for x in all_coords: 
        print(x)
        plot_polyiamond([x])
    # plot_polyiamond(all_coords)
    